In [ ]:
!pip3 install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.10.0-py3-none-any.whl size=205420 sha256=edceb4d59e8fd151d27447620dc43cb6cb46a9cf60dc5aec842c159074b9da34
  Stored in directory: /root/.cache/pip/wheels/ee/e3/f1/4c142d3be2e5f09c1ba8bf5801ae61d7b2cfd9afe497872806
Successfully built pyTelegramBotAPI


In [ ]:
import telebot
import requests

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp drive/MyDrive/HabourSpace/CapstoneProject/thetaleofkieu.h5 .
!cp drive/MyDrive/HabourSpace/CapstoneProject/clean.txt .

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras import regularizers

In [ ]:
vocab_size=4949
seq_length=60

In [ ]:
thetaleofkieu = Sequential()
thetaleofkieu.add(Embedding(vocab_size, 100, input_length=seq_length))
thetaleofkieu.add(Bidirectional(LSTM(150, return_sequences = True)))
thetaleofkieu.add(Dropout(0.2))
thetaleofkieu.add(LSTM(100))
thetaleofkieu.add(Dense(vocab_size/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
thetaleofkieu.add(Dense(vocab_size, activation='softmax'))
thetaleofkieu.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(thetaleofkieu.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 100)           494900    
                                                                 
 bidirectional (Bidirectiona  (None, 60, 300)          301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 60, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 2474)              249874    
                                                                 
 dense_1 (Dense)             (None, 4949)              12248775  
                                                        

In [ ]:
thetaleofkieu.load_weights('thetaleofkieu.h5')

In [ ]:
with open("clean.txt", "r") as file:
    data = file.read()

In [ ]:
token = Tokenizer()
clean_data=data.split("\n")
token.fit_on_texts(clean_data)

In [ ]:
poetry_length = 8
def generate_poetry(model,seed_text, n_lines):
  poem=[]
  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    poem.append(text)
  return " \n".join(poem)

In [ ]:
seed_text = 'courage'
text=generate_poetry(thetaleofkieu,seed_text, 5)

1/1 [==============================] - 0s 40ms/step


In [ ]:
print(text)

whom was he fell in love and you’ll 
know i’m back there together with the blue 
or gay is it up to our heart’s 
been strayed from home misstep after flirting butterflies 
were brimmed with transient grief with decked pain


In [ ]:
TELEGRAM_TOKEN = '6260866413:AAEIhmAhCxjPTCdg7CejjUku8QuprSY2vUs'

In [ ]:
import telebot

# create an instance of the bot
bot = telebot.TeleBot(TELEGRAM_TOKEN)

# define the start function
@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, "Hello! I'm your poem bot. Please enter the keyword?")
    bot.register_next_step_handler(message, handle_keyword)
# define the message handler function
@bot.message_handler(func=lambda message: True)
def handle_keyword(message):
    # send an echo message back to the user
    global keyword
    keyword=message.text
    #bot.send_message(message.chat.id,f" keyword: {keyword} is okay ")
    bot.send_message(message.chat.id,"please enter the length")
    bot.register_next_step_handler(message, handle_length)
@bot.message_handler(func=lambda message: True)
def handle_length(message):
    #keyword=keyword
    length=int(message.text)
    response = generate_poetry(thetaleofkieu,keyword, length)

    bot.send_message(message.chat.id, response)
    #bot.send_message(message.chat.id, message.text)

# start the bot
bot.polling()


1/1 [==============================] - 0s 42ms/step
